# Group 6
- GIL GUZMAN, ALEXSANDER GERARDO
- SALMÓN SALAZAR, GISELLA VELDA
- CIRIACO RUIZ, MAYTE JULIA
- ARAUCO ALIAGA, ROXANA PATRICIA
- VILLACORTA BARRERA, ANDRES ALEXANDER

In [1]:
# We import the necessary libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

1. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.ters.

In [ ]:
# We import the database
# We modified the format of the 'Ubigeo' and 'Casos' columns 
#to avoid loss of information

df = pd.read_csv('../../_data/data_dengue_peru.csv', 
                 dtype={'Ubigeo': 'str'}, 
                 converters={'Casos': lambda x: float(x.replace(',', '')) if x != '' else np.nan})

df

2. Generate ubigeo for Departments and Provinces taking the first two and four numbers. Hint: Use this code.

In [ ]:
# We generate the requested columns

df['Departamento_Ubigeo'] = df['Ubigeo'].str[:2]
df['Provincia_Ubigeo'] = df['Ubigeo'].str[:4]
df

3. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.

4. Use geopandas to plot the number of cases in 2021 by the province using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the province level.

5. Use geopandas to plot the number of cases by the department for all the years using subplots. Every subplot for each year. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the department level.

6. Use geopandas to plot the number of cases by the department for all 2021 quarters using subplots. Every subplot for each quarter. Use a categorical legend with 5 bins. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the department level. Hint: Use Semana variable to group by quarters.